<a href="https://colab.research.google.com/github/ZackPashkin/various-google-colab-snippets/blob/main/TorchAndroidModelQuanitize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare for Android. Torch Model Dynamic Quanitization

In [1]:
import torch
torch.__version__
# 1.8.1+cu101
# Python 3.7.10


'1.8.1+cu101'

In [ ]:
# example model , change to yours
from torchvision import models
model = models.resnet18(pretrained=True)
model.eval()

In [8]:
# simple dynamic quantization. Only linear layers
backend = "qnnpack"
model.qconfig = torch.quantization.get_default_qconfig(backend)
torch.backends.quantized.engine = backend
quantized_model = torch.quantization.quantize_dynamic(model.to("cpu"), {torch.nn.Linear}, dtype=torch.qint8)

In [10]:
# prepare for android
from torch.utils.mobile_optimizer import optimize_for_mobile
ts_model = torch.jit.script(model)
optimized_torchscript_model = optimize_for_mobile(ts_model)
optimized_torchscript_model.save('modelq2.pt')

In [ ]:
# test model
input_fp32 = torch.randn(4, 3, 224, 224)
preds = optimized_torchscript_model(input_fp32)
print(preds)